>>## Bagging algorithyms simple implementation

In [14]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as exp
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import BaggingClassifier         # different algo
from sklearn.ensemble import BaggingRegressor
from sklearn.ensemble import VotingClassifier     # multiple different algo.
from sklearn.ensemble import VotingRegressor
from sklearn.tree import plot_tree
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE     # to handle the imbalanced dataset
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

In [2]:
df = pd.read_csv('wine_data.csv')
df.head()

,Unnamed: 0,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


In [3]:
df = df.iloc[:,1:]
df.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


In [45]:
df.describe()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
count,6497.000000,6497.000000,6497.000000,6497.000000,6497.000000,6497.000000,6497.000000,6497.000000,6497.000000,6497.000000,6497.000000,6497.000000
mean,7.215307,0.339666,0.318633,5.443235,0.056034,30.525319,115.744574,0.994697,3.218501,0.531268,10.491801,5.818378
std,1.296434,0.164636,0.145318,4.757804,0.035034,17.749400,56.521855,0.002999,0.160787,0.148806,1.192712,0.873255
min,3.800000,0.080000,0.000000,0.600000,0.009000,1.000000,6.000000,0.987110,2.720000,0.220000,8.000000,3.000000
25%,6.400000,0.230000,0.250000,1.800000,0.038000,17.000000,77.000000,0.992340,3.110000,0.430000,9.500000,5.000000
50%,7.000000,0.290000,0.310000,3.000000,0.047000,29.000000,118.000000,0.994890,3.210000,0.510000,10.300000,6.000000
75%,7.700000,0.400000,0.390000,8.100000,0.065000,41.000000,156.000000,0.996990,3.320000,0.600000,11.300000,6.000000
max,15.900000,1.580000,1.660000,65.800000,0.611000,289.000000,440.000000,1.038980,4.010000,2.000000,14.900000,9.000000


In [52]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6497 entries, 0 to 6496
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   fixed acidity         6497 non-null   float64
 1   volatile acidity      6497 non-null   float64
 2   citric acid           6497 non-null   float64
 3   residual sugar        6497 non-null   float64
 4   chlorides             6497 non-null   float64
 5   free sulfur dioxide   6497 non-null   float64
 6   total sulfur dioxide  6497 non-null   float64
 7   density               6497 non-null   float64
 8   pH                    6497 non-null   float64
 9   sulphates             6497 non-null   float64
 10  alcohol               6497 non-null   float64
 11  quality               6497 non-null   int64  
dtypes: float64(11), int64(1)
memory usage: 609.2 KB


In [53]:
df.isnull().sum()

fixed acidity           0
volatile acidity        0
citric acid             0
residual sugar          0
chlorides               0
free sulfur dioxide     0
total sulfur dioxide    0
density                 0
pH                      0
sulphates               0
alcohol                 0
quality                 0
dtype: int64

In [56]:
df['quality'].value_counts()

6    2836
5    2138
7    1079
4     216
8     193
3      30
9       5
Name: quality, dtype: int64

In [4]:
x = df.drop(['quality'],axis=1)
y = df['quality']

In [61]:
y.value_counts()

6    2836
5    2138
7    1079
4     216
8     193
3      30
9       5
Name: quality, dtype: int64

In [5]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=10)

In [6]:
st =StandardScaler()
x_trainscaled = st.fit_transform(x_train)
x_testscaled = st.transform(x_test)

In [78]:
dtc = DecisionTreeClassifier()

In [79]:
dtc.fit(x_trainscaled,y_train)

DecisionTreeClassifier()

In [80]:
y_pred = dtc.predict(x_testscaled)

In [82]:
accuracy_score(y_test,y_pred)

0.5846153846153846

In [87]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           3       0.00      0.00      0.00         6
           4       0.18      0.11      0.14        53
           5       0.64      0.64      0.64       414
           6       0.61      0.64      0.62       549
           7       0.55      0.52      0.54       232
           8       0.39      0.41      0.40        44
           9       0.00      0.00      0.00         2

    accuracy                           0.58      1300
   macro avg       0.34      0.33      0.33      1300
weighted avg       0.58      0.58      0.58      1300



## hyperparameter turning

In [88]:
grid_param ={
    'criterion':['gini','entropy'],
    'max_depth':range(2,10,1),
    'min_samples_leaf':range(1,8,1),
    'min_samples_split':range(2,8,1),
    'splitter':['best','random']
}

In [89]:
grid = GridSearchCV(estimator=dtc,param_grid=grid_param,verbose=1,cv=3)

In [90]:
grid.fit(x_trainscaled,y_train)

Fitting 3 folds for each of 1344 candidates, totalling 4032 fits


GridSearchCV(cv=3, estimator=DecisionTreeClassifier(),
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': range(2, 10),
                         'min_samples_leaf': range(1, 8),
                         'min_samples_split': range(2, 8),
                         'splitter': ['best', 'random']},
             verbose=1)

In [93]:
grid.best_params_

{'criterion': 'gini',
 'max_depth': 8,
 'min_samples_leaf': 5,
 'min_samples_split': 2,
 'splitter': 'random'}

In [94]:
dtc2 = DecisionTreeClassifier(criterion='gini',max_depth=8,min_samples_leaf=5,min_samples_split=2,splitter='random')

In [95]:
dtc2.fit(x_trainscaled,y_train)

DecisionTreeClassifier(max_depth=8, min_samples_leaf=5, splitter='random')

In [99]:
y_pred2 = dtc2.predict(x_testscaled)

In [100]:
accuracy_score(y_test,y_pred2)

0.5315384615384615

In [101]:
print(classification_report(y_test,y_pred2))

              precision    recall  f1-score   support

           3       0.00      0.00      0.00         6
           4       0.43      0.06      0.10        53
           5       0.59      0.57      0.58       414
           6       0.50      0.76      0.60       549
           7       0.64      0.16      0.26       232
           8       0.50      0.02      0.04        44
           9       0.00      0.00      0.00         2

    accuracy                           0.53      1300
   macro avg       0.38      0.22      0.23      1300
weighted avg       0.55      0.53      0.49      1300



## Randomforestclassifier

In [20]:
rf = RandomForestClassifier()

In [103]:
rf.fit(x_trainscaled,y_train)

RandomForestClassifier()

In [105]:
y_pred3 = rf.predict(x_testscaled)

In [106]:
accuracy_score(y_test,y_pred3)

0.68

    ## RandomForestClassifier hyperameter

In [108]:
grid_param ={
    'n_estimators':[80,120,150,180,200],
    'criterion':['gini','entropy'],
    'max_depth':range(2,10,1),
    'min_samples_leaf':range(1,8,1),
    'min_samples_split':range(2,8,1),
    'oob_score':[True,False]
}

In [109]:
grid2 = GridSearchCV(estimator=rf,param_grid=grid_param,verbose=1,cv=3)

In [ ]:
grid2.fit(x_trainscaled,y_train)

Fitting 3 folds for each of 6720 candidates, totalling 20160 fits


In [ ]:
grid2.best_param_

In [ ]:
control userpasswords2

In [ ]:
rf2 = RandomForestClassifier(n_estimators=,criterion=,max_depth=,min_samples_leaf=,min_samples_split=,oob_score=)

## Bagging classifier with differnet algo.

In [15]:
lg = LogisticRegression()
svc = SVC()

In [9]:
bgc = BaggingClassifier(base_estimator=lg,n_estimators=20,oob_score=True)

In [16]:
bgc2 = BaggingClassifier(base_estimator=svc,n_estimators=20)

In [17]:
bgc2.fit(x_trainscaled,y_train)

BaggingClassifier(base_estimator=SVC(), n_estimators=20)

In [10]:
bgc.fit(x_trainscaled,y_train)

BaggingClassifier(base_estimator=LogisticRegression(), n_estimators=20,
                  oob_score=True)

In [11]:
y_pred = bgc.predict(x_testscaled)

In [12]:
accuracy_score(y_pred,y_test)

0.53

In [18]:
y_pred2 = bgc2.predict(x_testscaled)

In [19]:
accuracy_score(y_pred2,y_test)

0.5515384615384615

## VotingClassifier used for different algo.


In [25]:
## you can use best parameter for all these algorithym before specify here

vt = VotingClassifier(estimators=[('lr',lg),('sv',svc),('rf',rf)])

In [26]:
vt.fit(x_trainscaled,y_train)

VotingClassifier(estimators=[('lr', LogisticRegression()), ('sv', SVC()),
                             ('rf', RandomForestClassifier())])

In [27]:
y_pred3 = vt.predict(x_testscaled)

In [28]:
accuracy_score(y_pred3,y_test)

0.566923076923077

In [33]:
rf.fit(x_trainscaled,y_train)

RandomForestClassifier()

In [34]:
pred = rf.predict(x_testscaled)

In [35]:
accuracy_score(pred,y_test)          # which means there is DT is algorithm for the ensemble technique

0.6792307692307692

In [36]:
pred[1:10]

array([6, 6, 6, 5, 6, 6, 7, 6, 6], dtype=int64)

In [37]:
y_test[1:10]

2529    7
5029    6
5142    6
1551    5
5345    5
3234    6
3147    6
2289    4
6301    6
Name: quality, dtype: int64

## bagging technique completed